In [6]:
import pandas as pd
import json
import requests
try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode

from base64 import urlsafe_b64encode
import hashlib

def webshrinker_categories_v3(access_key, secret_key, url=b"", params={}):
    params['key'] = access_key

    request = "categories/v3/{}?{}".format(urlsafe_b64encode(url).decode('utf-8'), urlencode(params, True))
    request_to_sign = "{}:{}".format(secret_key, request).encode('utf-8')
    signed_request = hashlib.md5(request_to_sign).hexdigest()

    return "https://api.webshrinker.com/{}&hash={}".format(request, signed_request)

access_key = "u9BK7yp4mLJ2Ri5WNSBu"
secret_key = "IBfNwfA3iKz3VDVrW1E4"

categories = {}
df = pd.read_excel('michelin_data.xlsx')
for url in set(df['Attribute Value'].tolist()):
    if url != 'file:':
        req_url = url.encode()
        req = requests.get(webshrinker_categories_v3(access_key, secret_key, req_url)).json()
#         print(req_url)
        data = req['data']
        for key in data:
            for cat in key['categories']:
    #             print(cat['label'])
                categories[url] = cat['label']
                break
    else:
#         print(req_url)
        categories[url] = 'N/A'
dict_list = []
        
for dic in df.T.to_dict().values():
    dat = dic
    dat['category'] = categories[dat['Attribute Value']]
    dict_list.append(dat)
                      
df = pd.DataFrame.from_records(dict_list)

In [8]:
df.to_csv('michelin_axciom_data.csv', sep=',', encoding='utf-8')